Análise e Modelagem dos Dados Climáticos

In [ ]:
# --- Importações básicas e configs ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime

# Modelagem simples
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error, r2_score

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Caminhos para dados
RAW_DATA_DIR = Path('../data/raw')
PROCESSED_DATA_DIR = Path('../data/processed')


1. Carregando e Transformando Dados

In [14]:

def load_latest_json(data_dir):
    files = sorted(data_dir.glob("*.json"), reverse=True)
    if not files:
        raise FileNotFoundError("Nenhum arquivo JSON encontrado no diretório.")
    return pd.read_json(files[0], orient='records')

raw_df = load_latest_json(RAW_DATA_DIR)
print("Dados brutos carregados:")
display(raw_df.head())

def transform_raw_to_df(df):
    df = df.rename(columns={
        'temperature': 'temperature_celsius',
        'windspeed': 'wind_speed_kmh',
        'winddirection': 'wind_direction_degrees',
        'weathercode': 'weather_condition_code',
        'time': 'measurement_datetime',
        'interval': 'interval_seconds'
    })
    df['measurement_datetime'] = pd.to_datetime(df['measurement_datetime'])
    return df

df = transform_raw_to_df(raw_df)
print("Dados transformados:")
display(df.head())


Dados brutos carregados:


,latitude,longitude,generationtime_ms,utc_offset_seconds,timezone,timezone_abbreviation,elevation,current_weather_units,current_weather
time,-5.75,-35.25,0.048161,-10800,America/Fortaleza,GMT-3,36,iso8601,2025-06-28T17:00
interval,-5.75,-35.25,0.048161,-10800,America/Fortaleza,GMT-3,36,seconds,900
temperature,-5.75,-35.25,0.048161,-10800,America/Fortaleza,GMT-3,36,°C,24.7
windspeed,-5.75,-35.25,0.048161,-10800,America/Fortaleza,GMT-3,36,km/h,10.0
winddirection,-5.75,-35.25,0.048161,-10800,America/Fortaleza,GMT-3,36,°,142


KeyError: 'measurement_datetime'

2. Estatísticas Descritivas

In [9]:
print("Estatísticas descritivas gerais:")
display(df.describe())

print("Valores ausentes por coluna:")
print(df.isna().sum())


Estatísticas descritivas gerais:


NameError: name 'df' is not defined

3. Visualizações Básicas

In [ ]:
# Temperatura ao longo do tempo
sns.lineplot(data=df, x='measurement_datetime', y='temperature_celsius')
plt.title('Temperatura ao longo do tempo')
plt.xlabel('Data e Hora')
plt.ylabel('Temperatura (°C)')
plt.show()

# Distribuição velocidade do vento
sns.histplot(df['wind_speed_kmh'], bins=30, kde=True)
plt.title('Distribuição da Velocidade do Vento')
plt.xlabel('Velocidade do Vento (km/h)')
plt.show()

# Frequência condições climáticas (weather_condition_code)
plt.figure(figsize=(14,5))
sns.countplot(data=df, x='weather_condition_code', order=df['weather_condition_code'].value_counts().index)
plt.title('Frequência das condições climáticas')
plt.show()


4. Detecção de Anomalias Simples

In [ ]:
# Exemplo: temperatura muito alta ou muito baixa
temp_mean = df['temperature_celsius'].mean()
temp_std = df['temperature_celsius'].std()

# Considera anomalia valores fora de 3 desvios padrão
anomalias = df[(df['temperature_celsius'] > temp_mean + 3*temp_std) | (df['temperature_celsius'] < temp_mean - 3*temp_std)]
print(f"Encontradas {len(anomalias)} anomalias de temperatura:")
display(anomalias)


5. Preparação para Modelagem Preditiva

In [ ]:
# Criar features de tempo
df['hour'] = df['measurement_datetime'].dt.hour
df['dayofweek'] = df['measurement_datetime'].dt.dayofweek

# Selecionar colunas para modelo simples
features = ['hour', 'dayofweek', 'wind_speed_kmh', 'wind_direction_degrees']
target = 'temperature_celsius'

# Tratar valores ausentes, se houver
df = df.dropna(subset=features + [target])

X = df[features]
y = df[target]

# Dividir dados treino/teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


6. Treinamento e Avaliação do Modelo

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Métricas do modelo:")
print(f"RMSE: {root_mean_squared_error(y_test, y_pred, squared=False):.2f}")
print(f"R²: {r2_score(y_test, y_pred):.2f}")

# Plot real x previsto
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.xlabel("Temperatura Real (°C)")
plt.ylabel("Temperatura Prevista (°C)")
plt.title("Real x Previsto - Modelo Linear")
plt.show()


7. Salvando dados processados para análises futuras

In [ ]:
df.to_csv(PROCESSED_DATA_DIR / 'processed_weather_data.csv', index=False)
print(f"Dados processados salvos em: {PROCESSED_DATA_DIR / 'processed_weather_data.csv'}")
